<a href="https://colab.research.google.com/github/Meaveryway/Backend_AntTech/blob/master/1_Scraping_Msila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langdetect
import time
from datetime import datetime
import csv
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os.path
import re
from time import sleep
import math
from random import randint
import dateutil.parser as dateParser
import concurrent.futures

     |████████████████████████████████| 983kB 15.6MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=804dc736817ee7cc026f5c62f3b83fa146d3094c34e33ec4766af9c2fe431384
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [2]:
!pip freeze > requirements.txt

In [62]:
# Dataframes en mode interactif
%load_ext google.colab.data_table
#%unload_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


### Custom header 
for ethical scraping, in case some DSpace mod wants to contact me.

In [4]:
HEADERS = {
    'User-Agent': 'Meave Abdelhamid scraping bot',
    'From': 'gm_abdelhamid@esi.dz'
}

### Source details
for now I'm scraping only one repository but it should be the same process for all the others.
- Université Kasdi Merbah - Ouergla.

In [5]:
MSILA = {
    'base_url': "http://dspace.univ-msila.dz:8080",
    'institute_name': "Université Mohamed Boudiaf - M'sila"}

MSILA_TECHNOLOGIE = {'name': "MSILA_TECHNOLOGIE",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/14&rpp=100&page=',
                  'items_count': 1976,
                  'type': 'fr'}

MSILA_DROIT = {'name': "MSILA_DROIT",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/9&rpp=100&page=',
                  'items_count': 1672,
                  'type': 'ar'}

MSILA_MATHS = {'name': "MSILA_MATHS",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/15&rpp=100&page=',
                  'items_count': 801,
                  'type': 'fr'}

MSILA_LETTRES = {'name': "MSILA_LETTRES",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/11&rpp=100&page=',
                  'items_count': 3913,
                  'type': 'ar'}

MSILA_SCIENCES = {'name': "MSILA_SCIENCES",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/13&rpp=100&page=',
                  'items_count': 1233,
                  'type': 'fr'}

MSILA_ECONOMIE = {'name': "MSILA_ECONOMIE",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/12&rpp=100&page=',
                  'items_count': 2732,
                  'type': 'other'}

MSILA_SOCIALES = {'name': "MSILA_SOCIALES",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=',
                  'items_count': 4118,
                  'type': 'other_2'}

MSILA_URBA = {'name': "MSILA_URBA",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/8&rpp=100&page=',
                  'items_count': 655,
                  'type': 'other'}

MSILA_SPORT = {'name': "MSILA_SPORT",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/7&rpp=100&page=',
                  'items_count': 1354,
                  'type': 'other'}

MSILA_ = {'name': "",
                  'results_page_url' :'http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/14&rpp=100&page=',
                  'items_count': 0,
                  'type': 'idlery'}


In [ ]:
#Template
CITY = {
    'base_url': "http://....",
    'institute_name': 'Université ... - Ville'}
UNIVERSITY_TYPE_OR_FIELD = {'name': "UNIVERSITY_TYPE_OR_FIELD",
                  'results_page_url': "http://...start=",
                  'items_count': 0,
                  'type': "licence, doctorat, ... or idle si par discipline"}


# <font color="yellow"> **Setting-up the DSpace to scrape** </font>

In [128]:
current_dspace = MSILA 
repo_to_scrap = MSILA_ECONOMIE

In [129]:
DSPACE_BASE_URL = current_dspace['base_url']
INSTITUTE_NAME = current_dspace['institute_name']

# URL of the results page, set to display 100 elements to reduce the number of result pages we have to scrap overall.
RESULTS_PAGE_URL = repo_to_scrap['results_page_url']
FILE_NAME = repo_to_scrap['name']
ITEMS_NUMBER = repo_to_scrap['items_count']
FILE_TYPE = repo_to_scrap['type']

In [130]:
MAX_THREADS = 15 # Go with only 3 for Bejaia

### DSpace Fields
List of fields that will be scraped from DSpace as they appear in there. Details about fields are provided down there.

In [9]:
LABELS = ['dc.contributor.author', 'dc.subject', 'dc.description.abstract', 'dc.title', 'dc.identifier.uri', 'dc.date.issued', 'Appears in Collections:', 'Collection(s)\ :', "Collection(s) :"] # List 

# Access To Google Drive
In order to write files on there for persistency.

In [10]:
DRIVE_PATH = "/content/drive/MyDrive/Théses-Algérie/Data/"
MISSED_LINKS_PATH = DRIVE_PATH + "missingno"

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Standard & Drive file writing functions
*   rows for df -> csv is a list of items with the header row at [0]

In [12]:
def write_rows_to_sheet(ws, rows):
  ws.append_rows(rows)

def write_row_to_csv(file_path, row):
  with open(file_path, mode='a') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(row)

def write_rows_to_csv(file_path, rows):
  with open(file_path, mode='a') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerows(rows)

def write_df_to_csv(file_name, rows):
  df = pd.DataFrame.from_records(rows[1:], columns=rows[0])
  df.to_csv(file_name,index=False)  

In [13]:
# Write a DataFrame to CSV on drive
def write_df_on_drive(file_name, df):
  path = DRIVE_PATH + file_name
  df.to_csv(path)

# Read a DataFrame from a CSV on drive
def read_df_on_drive(file_name):
  path = DRIVE_PATH + file_name
  df = pd.read_csv(path)
  print("Read links from file:" + path)
  return df

# Move a file from Google drive to local runtime (for faster writings)
def move_csv_drive_local(file_name):
  read_df_on_drive()


# Append a missing link to the missingno file on drive. Missing links are ones that timeout.
def add_missingno(link):
  row = [link, INSTITUTE_NAME, FILE_TYPE]
  with open(MISSED_LINKS_PATH, mode='a') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(row)


# Language Detection
Used to detect the laguage based on the title, as language is a field in our database.

- LangDetect was found to be a better option since Tika is slower (runs an apache server transparently) but also strugles to recognise french. 

In [14]:
from langdetect import detect
title = 'ما هو لوريم ايبسوم  Lorem ipsum'
lang = detect(title)
print("Language = " + lang)

Language = ar


# ID generation
A pseudo-unique ID is given to each thesis element based an MD5 hash applied on its title. Only 16 first digits will be taken for the sake of simplicity and sufficience.

- Cases of ID collusions are either reposts of the exact same thesis (most times) or just lucky strikes (very rare but may happen in Master's cases). 
- Those will be dealt with at an epuration step before the AppSearch indexation. 


In [15]:
import hashlib
id_ = int(hashlib.md5(str(title).encode('utf-8')).hexdigest(), 16)
print("Id = " + str(id_))

Id = 179882734787083871587347073453909378217


# Scraping the results page
To get a list of urls for all documents listed (under "__items_url__") from the provided results page URL.
- Scraping one results page at a time and getting through them in increments that are listed on "__pages__". Default is a result page of a 100 element, with total number of elements provided as a repository detail up there.
- Moving between pages is done with just URL alterations at the moment.

# Results scraper (Standard for now)

### Functions

In [ ]:
def scrap_result_page(start_index):
  try:
    page_content = requests.get(RESULTS_PAGE_URL + str(start_index)).text
    soup = BeautifulSoup(page_content)
    print("Scraping page " + str(start_index) + " --> " + str(RESULTS_PAGE_URL + str(start_index)))
    for li in soup.find_all('li'):

      sup_div = li.find_all('div', {'class': 'artifact-description'}) #to exclude super categories at the top of page 1
      if len(sup_div) > 0:
        div = sup_div[0].find_all('div', {'class': 'artifact-title'})
        link = DSPACE_BASE_URL + div[0].find('a')['href']
          #print(link)
        write_row_to_csv(FILE_NAME, [link, -1])
  except Exception as e:
    print(e)

def scrap_result_pages(nb_elements):
  pages = np.arange(1, math.ceil(nb_elements/100)+1, 1)
  print(pages)
  print('---------------')
  write_row_to_csv(FILE_NAME, ['thesis_url', 'scraped'])
  #args = ((results_base_url, page, file_path) for page in pages)
  with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(scrap_result_page, pages)

In [ ]:
 def first_extraction():
  start = time.time()
  scrap_result_pages(ITEMS_NUMBER)
  scraping_time = time.time() - start

  print("Request completed in {0:.0f}s".format(scraping_time))

  #Writing to Drive
  df_urls = pd.read_csv(FILE_NAME)
  write_df_on_drive(FILE_NAME, df_urls)

### <font color="yellow">  **Launching call** </font>

In [ ]:
# Just to confirm which DSpace is being scraped
print(FILE_NAME)
print(ITEMS_NUMBER)

MSILA_SOCIALES
4118


In [ ]:
first_extraction()

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]
---------------
Scraping page 13 --> http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=13
Scraping page 1 --> http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=1
Scraping page 9 --> http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=9
Scraping page 11 --> http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=11
Scraping page 14 --> http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=14
Scraping page 3 --> http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=3
Scraping page 12 --> http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=12
Scraping page 7 --> http://dspace.univ-msila.dz:8080/xmlui/discover?scope=123456789/10&rpp=100&page=7
Scraping page 2 --> http://dspa

In [ ]:
# Preview
df_urls = pd.read_csv(DRIVE_PATH + FILE_NAME)
df_urls

,Unnamed: 0,thesis_url,scraped
0,0,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1
1,1,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1
2,2,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1
3,3,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1
4,4,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1
...,...,...,...
3713,3713,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1
3714,3714,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1
3715,3715,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1
3716,3716,http://dspace.univ-msila.dz:8080/xmlui/handle/...,-1




---

---



---



---



# Details scraper for <font color="chocolate"> **M'sila** </font>
Don't forget to replace all by "Articles Scientifiques Et Publications"

### Functions

In [144]:
def extract_thesis_details(link):
  try:
    #print(link + "?show=full") 
    element_link = requests.get(link + '?show=full', timeout=50).text
    element_soup = BeautifulSoup(element_link)
    #print(element_soup)
    
    item = {
        'id': '',
        'dc.title': '',
        'dc.contributor.author': [],
        'dc.description.abstract': '',
        'dc.subject': [],
        'type': '',
        'Collection(s) :': '', #field
        'language': '',
        'dc.date.issued': '',
        'dc.identifier.uri': '',
        'institute_name': '',
        
        'file_url': '',
        'file_size': ''
        }

    #extracting thesis details
    table = element_soup.find('table', {'class': 'ds-includeSet-table detailtable'})
    for tr in table.find_all('tr'):
      label = tr.find('td', {'class': 'label-cell'}).text
      value = tr.find_all('td')[1].text
      #print("Value = {} for: {}".format(value, label))
      if('llection' in label): #capturing the 'Collection(s) :' field (caused problems)
        item['Collection(s) :'] = value
      if(label in LABELS):
        if type(item[label]) is list:
          item[label].append(value)
        else:
          item[label] = value
    #extracting file details
    file_url_div = element_soup.find('div', {'class': 'file-link'})
    file_url = DSPACE_BASE_URL + file_url_div.find('a')['href']
    file_size = element_soup.find('div', {'class': 'file-metadata'}).find_all('div')[1].find_all('span')[1].text
    
    item['file_url'] = file_url
    item['file_size'] = file_size
    #filling in rest of data
    if len(item['dc.description.abstract']) > 10:
      item['language'] = detect(item['dc.description.abstract'])
    else:
      item['language'] = detect(item['dc.title'].lower())
    item['id'] = int(str(int(hashlib.md5(str(item['dc.title']).encode('utf-8')).hexdigest(), 16))[:16])
    item['institute_name'] = institute_name
    item['dc.identifier.uri'] = link

    #getting field and type from breadcrumb in top of page
    breadcrumb = element_soup.find_all('li', {'class': 'ds-trail-link'})
    #item['Collection(s) :'] = breadcrumb[2].text
    #doc_type = breadcrumb[3].text
    
    # for faculties in french
    if FILE_TYPE == "fr":
      item['Collection(s) :'] = breadcrumb[int(len(breadcrumb)-3)].text
      doc_type = breadcrumb[len(breadcrumb)-2].text

      item['type'] = "Thèse de Doctorat" if 'doctorat' in doc_type.lower() else "Mémoire de Master" if 'master' in doc_type.lower() else "Mémoire de Magister" if 'magister' in doc_type.lower() else "Mémoire de Licence" if 'licence' in doc_type.lower() else "Mémoire d'Ingéniorat" if 'ing' in doc_type.lower() else "Articles Scientifiques" if "article" in doc_type.lower() else doc_type.lower()

    #for faculties in arabic
    elif FILE_TYPE == "ar":

      breadcrumb_text = " == ".join([x.text.replace('\n', '') for x in breadcrumb[1:len(breadcrumb) - 1]])
      item['Collection(s) :'] = breadcrumb_text
      doc_type = breadcrumb_text

      item['type'] = "Thèse de Doctorat" if 'دكتور' in doc_type.lower() else "Mémoire de Master" if 'ماستر' in doc_type.lower() else "Mémoire de Magister" if 'ماج' in doc_type.lower() else "Mémoire de Licence" if 'ليس' in doc_type.lower() else "Mémoire d'Ingéniorat" if 'ing' in doc_type.lower() else "Articles Scientifiques" if "مقال" in doc_type.lower() else doc_type.lower()

    elif FILE_TYPE == "other":
      breadcrumb_text = " == ".join([x.text.replace('\n', '') for x in breadcrumb[1:len(breadcrumb) - 1]])
      item['Collection(s) :'] = breadcrumb_text
      doc_type = breadcrumb_text

      item['type'] = "Thèse de Doctorat" if 'دكتور' in doc_type.lower() else "Mémoire de Master" if 'ماستر' in doc_type.lower() else "Mémoire de Magister" if 'ماج' in doc_type.lower() else "Mémoire de Licence" if 'ليس' in doc_type.lower() else "Mémoire d'Ingéniorat" if 'ing' in doc_type.lower() else "Articles Scientifiques" if "مقال" in doc_type.lower() else doc_type.lower()

    elif FILE_TYPE == "other_2":
      breadcrumb_text = " == ".join([x.text.replace('\n', '') for x in breadcrumb[1:len(breadcrumb) - 1]])
      item['Collection(s) :'] = breadcrumb_text
      doc_type = breadcrumb_text

      item['type'] = "Thèse de Doctorat" if 'دكتور' in doc_type.lower() else "Mémoire de Master" if 'ماستر' in doc_type.lower() else "Mémoire de Magister" if 'ماج' in doc_type.lower() else "Mémoire de Licence" if 'ليس' in doc_type.lower() else "Mémoire d'Ingéniorat" if 'ing' in doc_type.lower() else "Articles Scientifiques" if "مقال" in doc_type.lower() else doc_type.lower()

    #writing item to CSV file
    write_row_to_csv(file_path_details, list(item.values()))
    print(datetime.now().strftime("%H:%M:%S") + ' = ' + str(item['dc.title'])[:75])
    return list(item.values())
  
  except Exception as e: 
    print("\033[93m Error - failed to scrap url: \033[0m")
    print(link)
    print(e)
    add_missingno(link=link)
    return []

In [145]:
def second_extraction(links, file_path):
    path = os.path.isfile(file_path)
    if (not path):
      print("\033[94m Creating new CSV file \033[0m")  
      write_row_to_csv(file_path, [
      'id',
      'title',
      'author',
      'abstract',
      'keywords',
      'type',
      'field',
      'language',
      'date_issued',
      'identifier_uri',
      'institute_name',
      'file_url',
      'file_size'])
    #with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
      #executor.map(extract_thesis_details, links)
    with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_THREADS) as executor:
      start = time.time()
      futures = [ executor.submit(extract_thesis_details, link) for link in links ]
      results = []
      for result in concurrent.futures.as_completed(futures):
        results.append(result)
      end = time.time()
      print("\033[92m Time Taken for all processes: {:.6f}s \033[0m".format(end-start))
    

In [146]:
def meta_second_extraction(links, file_name, chunk_size):
  chunks = np.arange(0, len(links), chunk_size)
  print(chunks)
  for chunk in chunks:
    chunk_start = chunk
    chunk_end = (chunk_start + chunk_size) if (chunk_start + chunk_size < len(links)) else (len(links))
    chunk_links = links[chunk_start:chunk_end]
    print("\033[92m" + datetime.now().strftime("%H:%M:%S") + " Scraping chunk: " + str(chunk_start) + " - " + str(chunk_end) + "\033[0m")
    second_extraction(chunk_links, file_name)
    sleep(randint(1,2))


### <font color="yellow">  **Launching call** </font>

In [147]:
# Just to confirm which DSpace is being scraped
file_path_details = FILE_NAME + "_DETAILS"
institute_name = INSTITUTE_NAME

In [148]:
FILE_NAME

'MSILA_ECONOMIE'

In [ ]:
CHUNK_SIZE = 80
# reading links from base CSV on drive
df_urls = read_df_on_drive(FILE_NAME)
print(len(df_urls))
links = df_urls['thesis_url']
#links = links[:1]

start = time.time()
meta_second_extraction(links, FILE_NAME + "_DETAILS", CHUNK_SIZE)
scraping_time = time.time() - start

print("\033[92m Request completed in {0:.0f}s \033[0m".format(scraping_time))
print("chunk size = " + str(CHUNK_SIZE))
print("number of scraped elements = " + str(len(links)))

### <font color="yellow"> **Post Processing** </font>

In [150]:
df_details = pd.read_csv(FILE_NAME + '_DETAILS')
df_details['field'] = df_details['field'].str.lower().str.replace("  ", " ").str.replace('departement des ','').str.replace('departement de ','').str.replace('departement d\'','').str.replace('département ','').str.replace('département des ','').str.replace('département de ','').str.replace('département d\'','').str.replace(' - doctorat', '').str.replace(' - magister', '').str.replace(' - master', '')
df_details['field'] = df_details['field'].str.lower().str.replace("faculté des", "").str.replace("faculté de", "").str.replace("faculté d'", "").str.replace("et des", "et").str.replace("et de", "et").str.replace("et d'", "et"). str.title()
df_details['field'] = df_details['field'].str.replace("\n", "")

df_details['type'] = df_details['type'].str.replace("\n", "")

df_details['date_issued'] = df_details['date_issued'].astype(str) #to avoid cases where it is actually properly documented as year only
df_details['date_issued'] = df_details['date_issued'].str.replace('nan', '3000')
df_details['date_issued'] = df_details['date_issued'].apply(lambda x: dateParser.parse(x[:10]).year) 

#cleaning "promoteur/rice" from authors names
df_details['author'] = df_details['author'].str.lower().str.replace(" ; ", "").str.replace(" ;", "").str.replace("; ", "").str.replace(";", "").str.replace(":", "").str.replace("promoteur", "").str.replace("promotrice", "").str.replace('()', '').str.title()

#Removing . if last char: 

df_details['keywords'] = df_details['keywords'].str[2:-2].apply(lambda x: "[$" + x + "$]") 
df_details['keywords'] = df_details['keywords'].str.lower().str.replace("mots-cles : ", "").str.replace("keywords: ", "").str.replace("الكلمات المفتاحية : ", "").str.replace("الكلمات المفتاحية: ", "").str.replace("الكلمات المفتاح: ", "")
df_details['keywords'] = df_details['keywords'].str.replace(".", ",").str.replace('.', ',').str.replace("، ", ",").str.replace("،", ",").str.replace(" ،", ",").str.replace('/', ",")
df_details['keywords'] = df_details['keywords'].str.replace(";", ",").str.replace(":", ",").str.replace('- ', ',').str.replace(' -', ',').str.replace(" – ", ",")
df_details['keywords'] = df_details['keywords'].str.replace(", ", ",").str.replace(" ,", ",").str.replace("','", ",").str.replace(",", "$,$")#.str.replace('"', "$")

#df_details['type'] = "Mémoire de Doctorat"

#removing empty keywords
df_details['keywords'] = df_details['keywords'].str.replace("\,\$\$", "").str.replace('\$\$', '')

#write_df_on_drive(FILE_NAME + '_DETAILS', df_details)


In [167]:
df_details['field'] = df_details['field'].apply(lambda x: "Sciences Économiques Et Commerciales Et Sciences De Gestion" if "iences Economiques Et Commerciales, Sciences " in x else x)
#df_details['type'] = df_details['type'].apply(lambda x: "Articles Scientifiques Et Publications" if "Articles" in x else x)

In [168]:
df_details_keywords = df_details
print(df_details_keywords.field.unique())
print('--------------------------------')
print(df_details_keywords.type.unique())

['Sciences Commerciales' 'Gestion' 'Économie' 'Finances Et Comptabilité'
 'Sciences Économiques Et Commerciales Et Sciences De Gestion']
--------------------------------
['Mémoire de Master' 'Mémoire de Magister' 'Thèse de Doctorat'
 'Articles Scientifiques Et Publications']


In [ ]:
# Preview
df_details_keywords = df_details.loc[:, df_details.columns != 'abstract']
df_details_keywords = df_details_keywords.loc[:, df_details.columns != 'title']
df_details_keywords

In [169]:
write_df_on_drive(FILE_NAME + '_DETAILS', df_details)



---



---



---



---

